# Processamento Digital de Imagens

1001527 - Turma A

Prof. Cesar Henrique Comin

## Sistema para desenhar no ar com um objeto colorido

### Equipe
Igor Teixeira Machado RA: 769708

Rafael Vinícius Passador RA: 790036

* pip install opencv-python
* pip install numpy

In [22]:
import cv2
import numpy as np

In [23]:
def detect_object(frame):

    # Convertendo para HSV.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Escolhendo limites para a detecção da cor do objeto vermelho.
    #lower_bound = np.array([10, 100, 20])
    #upper_bound = np.array([25, 255, 255])

    # Escolhendo limites para a detecção da cor do objeto azul.
    lower_bound = np.array([90, 50, 70])
    upper_bound = np.array([128, 255, 255])

    # Aplicando os limites.
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Definindo o tamanho do kernel.
    kernel = np.ones((7, 7), np.uint8)

    # Removendo ruídos.
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    return mask

def mass_center(mask, img):

    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        c = max(contours, key=cv2.contourArea)
        matrix = cv2.moments(c)

        if matrix["m00"] != 0:
            cx = int(matrix["m10"] / matrix["m00"])
            cy = int(matrix["m01"] / matrix["m00"])
            center = (cx, cy)
            
            return center
    
    return None


def draw_line(image, start_point, end_point, radius=5, color=(0, 0, 255), thickness=3):
    
    # cor: BGR
    image = cv2.line(image, start_point, end_point, color, thickness)

    return image

def draw_circle(image, center, radius=5, color=(0, 0, 255), thickness=-1):
        
    # cor: BGR
    image = cv2.circle(image, center, radius, color, thickness)

    return image

In [24]:
# Abre dispositivo de captura. O parâmetro 0 faz com que
# a primeira câmera encontrada seja utilizada
vcap = cv2.VideoCapture(0)
points = []

while True:
    ret, curr_frame = vcap.read()
    if ret:
        # Redimensionamos a imagem para diminuir o custo computacional
        # necessário para processar o frame
        resized_frame = cv2.resize(curr_frame, (640, 360))
        resized_frame = cv2.flip(resized_frame, 1) # Espelhamento horizontal
        paint_object = detect_object(resized_frame)
        object_mass_center = mass_center(paint_object, resized_frame)

        if object_mass_center is not None:
            points.append(object_mass_center)

        # Definindo o tamanho maximo da lista de pontos ou se nao houver mais objetos detectados apagamos a linha
        if len(points) > 100 or object_mass_center is None and len(points) > 0:
            points.pop(0)

        for i in range(len(points) - 1):
            start_point = points[i]
            end_point = points[i + 1]
            resized_frame = draw_line(resized_frame, start_point, end_point)
               
        cv2.imshow('frame', resized_frame)
    else:
        print("Frame not available")

    # Esperar 33 milisegundos significa mostrar 30 imagens por segundo (1/0.033).
    # Portanto, o FPS (frames per second) do vídeo mostrado será aproximadamente 30.
    # O valor não é exatamente 30 porque um tempo é gasto no processamento do frame
    key = cv2.waitKey(33) & 0xff

    if key == 27:
        break

vcap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1